# HRRR Forecast Collection Best Time Series 
This notebook demonstrates how to access a collection of GRIB2 files as a single cloud-friendly dataset in xarray. 

We first use [kerchunk](https://github.com/fsspec/kerchunk) to create the JSON file that is then read by the zarr libaray using a referenceFileSystem created by fsspec.   


In [ ]:
import warnings
warnings.simplefilter("ignore")  # filter some warning messages

In [ ]:
import xarray as xr
import hvplot.xarray
import datetime as dt
import pandas as pd
import dask
import panel as pn
import json
import fsspec
from kerchunk.grib2 import scan_grib
from kerchunk.combine import MultiZarrToZarr

#### Create a best time series dataset
There is a new HRRR forecast every hour, so use forecast hour 1 from 
past forecasts and then append the latest forecast

In [ ]:
fs = fsspec.filesystem('s3', anon=True, skip_instance_cache=True)

In [ ]:
today = dt.datetime.utcnow().strftime('%Y%m%d')

In [ ]:
last_file = fs.glob(f's3://noaa-hrrr-bdp-pds/hrrr.{today}/conus/*wrfsfcf01.grib2')[-1]

In [ ]:
last_file

In [ ]:
end_time = dt.datetime.strptime(last_file,
        'noaa-hrrr-bdp-pds/hrrr.%Y%m%d/conus/hrrr.t%Hz.wrfsfcf01.grib2')

In [ ]:
start_time = end_time - dt.timedelta(days=7)

In [ ]:
last_file

In [ ]:
glob_pattern_for_latest_forecast = end_time.strftime('s3://noaa-hrrr-bdp-pds/hrrr.%Y%m%d/conus/hrrr.t%Hz.wrfsfcf*.grib2')
print(glob_pattern_for_latest_forecast)

In [ ]:
dates = pd.date_range(start=start_time, end=end_time, freq='1h')

In [ ]:
dates

In [ ]:
files = [date.strftime('s3://noaa-hrrr-bdp-pds/hrrr.%Y%m%d/conus/hrrr.t%Hz.wrfsfcf01.grib2') for date in dates]

latest_files = fs.glob(glob_pattern_for_latest_forecast)
latest_files = [f's3://{file}' for file in latest_files]
print(latest_files[-1])

In [ ]:
latest_files

In [ ]:
files.extend(latest_files[2:])
print(files[0])
print(files[-1])

In [ ]:
files

#### Create Dask gateway cluster with credentials to write to specified S3 bucket
Unless you are using the ESIP Qhub, you will need to modify the following cell to create a Dask cluster and supply your Dask workers with the necessary credentials to write to the S3 bucket.  Or modify to use a local Dask cluster and write JSON to local disk.   

In [ ]:
import os
import sys
sys.path.append(os.path.join(os.environ['HOME'],'shared','users','lib'))
import ebdpy as ebd

profile = 'esip-qhub'
region = 'us-west-2'
endpoint = f's3.{region}.amazonaws.com'

ebd.set_credentials(profile=profile, region=region, endpoint=endpoint)
worker_max = 30
client,cluster = ebd.start_dask_cluster(profile=profile,worker_max=worker_max, 
                                      region=region, use_existing_cluster=True,
                                      adaptive_scaling=False, wait_for_cluster=True, 
                                      environment='pangeo', worker_profile='Medium Worker', 
                                      propagate_env=True)

#### Create individual ReferenceFileSystem jsons
The `afilter` below controls which grib variables we want [cfgrib](https://github.com/ecmwf/cfgrib#filter-heterogeneous-grib-files) to capture

In [ ]:
afilter={'typeOfLevel': 'heightAboveGround', 'level': 2}    

In [ ]:
so = {"anon": True, "default_cache_type": "readahead"}
common = ['time', 'step', 'latitude', 'longitude', 'valid_time']

In [ ]:
def gen_json(u):
    date = u.split('/')[3].split('.')[1]
    name = u.split('/')[5].split('.')[1:3]
    outfname = f'{json_dir}{date}.{name[0]}.{name[1]}.json'
    out = scan_grib(u, common, so, inline_threashold=100, filter=afilter)        
    with fs2.open(outfname, "w") as f:
        f.write(json.dumps(out))

#### Bucket to store individual JSONs (need permission, so anon=False)

In [ ]:
json_dir = 's3://esip-qhub/noaa/hrrr/jsons/'
fs2 = fsspec.filesystem('s3', anon=False, skip_instance_cache=True) 

In [ ]:
try:
    fs2.rm(json_dir, recursive=True)
except:
    pass

#### Compute the individual JSONs in parallel by computing a list of Dask delayed objects

In [ ]:
%%time
_ = dask.compute(*[dask.delayed(gen_json)(u) for u in files], retries=10);

#### Use `MultiZarrToZarr()` to combine into single reference

In [ ]:
flist2 = fs2.ls(json_dir)
furls = sorted(['s3://'+f for f in flist2])
print(furls[0])
print(furls[-1])

In [ ]:
furls

In [ ]:
mzz = MultiZarrToZarr(furls, 
            storage_options={'anon':False},
            remote_protocol='s3', 
            remote_options={'anon': True},
            xarray_concat_args={'dim': 'valid_time'})

In [ ]:
%%time
#%%prun -D multizarr_profile 
mzz.translate('hrrr_best.json')

In [ ]:
rpath = 's3://esip-qhub-public/noaa/hrrr/hrrr_best.json'
fs2.put_file(lpath='hrrr_best.json', rpath=rpath)

## Examine resulting dataset

In [ ]:
rpath = 's3://esip-qhub-public/noaa/hrrr/hrrr_best.json'
s_opts = {'requester_pays':True, 'skip_instance_cache':True}
r_opts = {'anon':True}
fs = fsspec.filesystem("reference", fo=rpath, ref_storage_args=s_opts,
                       remote_protocol='s3', remote_options=r_opts)
m = fs.get_mapper("")

ds2 = xr.open_dataset(m, engine="zarr", backend_kwargs=dict(consolidated=False), 
                      chunks={'valid_time':1})

In [ ]:
fs.ls('')

In [ ]:
var='t2m'

In [ ]:
print(f'Size:{ds2[var].nbytes/1e9} GB')

In [ ]:
ds2

Hvplot wants lon [-180,180], not [0,360]:

In [ ]:
ds2 = ds2.assign_coords(longitude=(((ds2.longitude + 180) % 360) - 180))

In [ ]:
now = dt.datetime.utcnow().strftime('%Y-%m-%d %H:00:00')
print(now)

With 30 worker cluster, takes 50 seconds to display, and 15 seconds to change the time step
after closing the dask client, it takes 30 seconds to display, 8 seconds to display a time step

In [ ]:
ds2[var].sel(valid_time=now).hvplot.quadmesh(x='longitude', y='latitude', geo=True,
                                                rasterize=True, cmap='turbo', title=now)

hvplot has a slider for time steps, but we want a dropdown list, so we use Panel.  Let's add a tile layer from Open Street Map also. 

In [ ]:
extra_dims = list(ds2[var].dims[:-2])
mesh = ds2[var].hvplot.quadmesh(x='longitude', y='latitude', rasterize=True, geo=True, tiles='OSM', cmap='turbo')
pn.panel(mesh, widgets={k: pn.widgets.Select for k in extra_dims})

#### Extract a time series at a point
We are reading GRIB2 files, which compress the entire spatial domain as a single chunk.  Therefore reading all the time values at a single point actually needs to load and uncompress *all* the data for that variable. 

In [ ]:
%%time
ds2[var][:,500,500].hvplot(x='valid_time', grid=True)

#### Compute the time-mean temperature over the entire dataset

In [ ]:
%%time
da = ds2[var].mean(dim='valid_time').compute()

In [ ]:
da.hvplot.quadmesh(x='longitude', y='latitude', rasterize=True, geo=True, tiles='OSM', cmap='turbo')

#### Close the client and shutdown the cluster

In [ ]:
client.close(); cluster.shutdown()